In [ ]:
%pylab notebook
from glob import glob
import json
import os.path

π = np.pi

In [ ]:
dr6_runs = glob(os.path.expanduser('~/SIMU/20200124/dynamic_run_6_*.json')) ; dr6_runs.sort()

with open('/home/pierre/code/RMPCDMD/experiments/04-enzyme/pfiles_dynamic_run_6/dynamic_run_6_20200122_00002.json', 'r') as f:
    input_data = json.load(f)                                     


In [ ]:
all_rates_data = [] 
for fn in dr6_runs: 
    with open(fn, 'r') as f:
        data = json.load(f)
    all_rates_data.append(data)


In [ ]:
e_sim_dt = np.dtype([
    ('n_e0', float),
    ('phi', float),
    ('n_fluid', float),
    ('n_s', float),
    ('n_p', float),
    ('n_estar', float),
    ('n_e', float),
    ('n_s_plus', float),
    ('n_p_plus', float),
    ('n_s_minus', float),
    ('n_p_minus', float),
])

In [ ]:
es = np.zeros(4, dtype=e_sim_dt)

In [ ]:
n_solvent = np.array([d['n_solvent']['mean'] for d in all_rates_data])
n_minus = np.array([d['n_minus']['mean'] for d in all_rates_data])
n_plus = np.array([d['n_plus']['mean'] for d in all_rates_data])

ρ = 12                                             
V = 30**3
v_e = 4/3*pi * (3**3 + 2)
substrate_fraction = float(input_data['substrate_fraction'])
ρ = int(input_data['rho'])

all_n_e0 = np.array([10, 20, 30, 40])

for i, d in enumerate(all_rates_data):
    n_e0 = d['N_enzymes']
    ϕ = n_e0 * v_e / V
    n_fluid = ρ * (V - n_e0*v_e)
    n_solvent = np.array(d['n_solvent']['mean'])
    n_plus = np.array(d['n_plus']['mean'])
    n_minus = np.array(d['n_minus']['mean'])
    n_estar = (n_fluid*substrate_fraction - n_solvent.sum())
    n_e = n_e0 - n_estar
    es[i]['n_e0'] = n_e0
    es[i]['n_e'] = n_e
    es[i]['n_estar'] = n_estar

    es[i]['phi'] = ϕ
    es[i]['n_fluid'] = n_fluid
    es[i]['n_s'] = n_solvent[0]
    es[i]['n_p'] = n_solvent[1]
    es[i]['n_s_plus'] = n_plus[0]
    es[i]['n_p_plus'] = n_plus[1]
    es[i]['n_s_plus'] = n_plus[0]
    es[i]['n_p_plus'] = n_plus[1]



In [ ]:
es['n_s']

In [ ]:
plt.figure()

plt.plot(es['phi'], es['n_estar'] / es['n_e'] / es['n_s'])
plt.plot(es['phi'], es['n_estar'] / es['n_e'] / es['n_s'] * (1 - es['phi']))


In [ ]:
R = float(input_data['sigma_E'])*2**(1/6)    
V = np.product(list(map(float, input_data['L'].split())))

sigma_N = list(map(float, input_data['sigma_N'].split()))

k1_0 = float(input_data['proba_s']) * np.sqrt(8*π*float(input_data['T'])) * R**2
k2_0 = float(input_data['proba_p']) * np.sqrt(8*π*float(input_data['T'])) * R**2
km1 = float(input_data['rate_release_s'])
km2 = float(input_data['rate_release_p'])
kappa = km1+km2

for i in range(4):
    n_e0 = es[i]['n_e0']
    e0 = n_e0 / V
    v_real = V - n_e0 * v_e
    s0 = es[i]['n_fluid'] / V
    phi = es[i]['phi']

    # Solve

    γ = 1/(1-phi)
    k1 = k1_0*γ
    k2 = k2_0*γ
    μ = k1/(1+k1*km2/km1/k2) + k2/(km1*k2/k1/km2 + 1)
    Δ = (μ*(e0+s0) + kappa)**2 - 4*μ**2*s0*e0

    estar = (( μ*(s0+e0)+kappa) - np.sqrt(Δ)) / (2*μ)
    S = (s0-estar) / (1+k1*km2/km1/k2)
    P = (s0-estar) / (km1*k2/k1/km2 + 1)
    
    #print(estar, es[i]['n_estar']/V)
    print(es[i]['n_s']/V, S)
